In [1]:
import os, sys
import re
import numpy as np
import imageio as im
import pandas as pd
import cv2
from skimage.morphology import skeletonize
from skimage.util import invert
from IPython.display import Image
import matplotlib.pyplot as plt
from cv2 import imwrite
from cv_algorithms import grassfire
from cv_algorithms import thinning as ta
from cv_algorithms.morphology import difference_of_gaussian
from fil_finder import FilFinder2D
from astropy.io import fits
import astropy.units as u
from IPython.core.display import HTML
cell_path = "individual_cells/"

In [4]:
def fskel(mask, b_thresh=40, sk_thresh=20):
    fil=FilFinder2D(mask,mask=mask)
    fil.preprocess_image(skip_flatten=True)
    fil.create_mask(use_existing_mask=True)
    fil.medskel(verbose=False)
    unpruned_skel = fil.skeleton
    fil.analyze_skeletons(branch_thresh=b_thresh*u.pix, skel_thresh=sk_thresh*u.pix, prune_criteria='length')
    skel = fil.skeleton_longpath
    return unpruned_skel,skel

In [5]:
all_the_cells = os.listdir(cell_path)

# Create a list that records the timepoints for every mask. 
time_list = []
# List of masks with associated timepoints and names
maskl = []
#list of skeletons (before and after branch pruning) with associated timepoints
unpruned_skell=[]
skell = []
#list of masks with skeletons overlayed in black
overl=[]
#overwrite skeleton_check
for item in os.listdir('skeleton_check'):
    if item[-3]=='p' and item[-2]=='n' and item[-1]=='g':
        os.remove('skeleton_check'+'/'+item)
for cell in all_the_cells:
    time = re.findall(r'\d+', cell)
    if time != []:
        #overwrite old skeletons in individual_cells
        for item in os.listdir(cell_path+cell+'\skeletons'):
            if item[-3]=='p' and item[-2]=='n' and item[-1]=='g':
                os.remove(cell_path+cell+'\skeletons'+'/'+item)
        time = time[0]
        time_list.append(int(time))
        im_path = cell_path + cell + '\masks'
        #create skeletons; save masks, skeletons, and masks with skeletons as lists, save skeletons and skeletons with masks as images
        for mask_name in os.listdir(im_path):
            mask=im.imread(im_path+'/'+mask_name)[:,:,0]>0 #the cells mask (boolean array)
            unpruned_skel, skel = fskel(mask*255)
            over = invert(skel+invert(mask)) #the mask overlayed on the skeleton (boolean array)
            maskl.append([mask,time,int(mask_name[5:6])])
            unpruned_skell.append([unpruned_skel,time,int(mask_name[5:6])])
            skell.append([skel,time,int(mask_name[5:6])])
            overl.append([over,time,int(mask_name[5:6])])
            imwrite(cell_path + cell+'\skeletons' + '\pruned_'+ mask_name[5:6]+'.png',255*skel)
            #imwrite(cell_path + cell+'\skeletons' + '\unpruned_' + mask_name[5:6]+'.png',255*unpruned_skel)
            imwrite(cell_path + cell+'\skeletons' + '\overlay_'+ mask_name[5:6]+'.png',255*over)

In [11]:
fil=FilFinder2D(maskl[0][0],mask=maskl[0][0])
help(fil.analyze_skeletons)

Help on method analyze_skeletons in module fil_finder.filfinder2D:

analyze_skeletons(prune_criteria='all', relintens_thresh=0.2, nbeam_lengths=5, branch_nbeam_lengths=3, skel_thresh=None, branch_thresh=None, max_prune_iter=10, verbose=False, save_png=False, save_name=None) method of fil_finder.filfinder2D.FilFinder2D instance
    Prune skeleton structure and calculate the branch and longest-path
    lengths. See `~Filament2D.skeleton_analysis`.
    
    
    Parameters
    ----------
    prune_criteria : {'all', 'intensity', 'length'}, optional
        Choose the property to base pruning on. 'all' requires that the
        branch fails to satisfy the length and relative intensity checks.
    relintens_thresh : float, optional
        Relative intensity threshold for pruning. Sets the importance
        a branch must have in intensity relative to all other branches
        in the skeleton. Must be between (0.0, 1.0].
    nbeam_lengths : float or int, optional
        Sets the minimum s